חלק א': הכנת הנתונים והבנת הנתונים של טבלת המכירות

ייבוא וארגון הדאטה בפייתון

1.import the csv files 

In [331]:
import pandas as pd
import calendar

# Paths to your CSV files
file1 = './Products.csv'
file2 = './Transactions.csv'

2.read the csv

In [332]:
# Read the CSV files
Products = pd.read_csv(file1)
Transactions = pd.read_csv(file2)

הבנת מבנה הנתונים:


1.txt תשובה על סעיף 1 בקובץ
2.txt תשובה על סעיף 2 בקובץ
3.txt תשובה על סעיף 3 בקובץ

איחוד הטבלאות:


1.Merge tables using the Product field


In [333]:

# Merge tables using the Product field
df_transactions = pd.DataFrame(Transactions)
df_Products = pd.DataFrame(Products)
df_transactions.columns = df_transactions.columns.str.strip()
df_Products.columns = df_Products.columns.str.strip()
enriched_data = pd.merge(df_transactions, df_Products, on="Product", how="left")
df = pd.DataFrame(enriched_data)
output_filename = "enriched_data.csv"
df.to_csv(output_filename, index=False)

2.Split the 'Segement_Country' column into 'Segment' and 'Country' and drop the old column

In [334]:
# Split the 'Segement_Country' column into 'Segment' and 'Country'
df[['Segment', 'Country']] = df['Segement_Country'].str.split('-', expand=True)
# Drop the original 'Segement_Country' column if it's no longer needed
df = df.drop(columns=['Segement_Country'])
output_filename = "drop_Segement_Country.csv"
df.to_csv(output_filename, index=False)

בדיקת תקינות:


1,2.checking if the merge was ok

In [335]:
def validate_merge(df_transactions, df_products, merged_df, key_column="Product"):
    # Check if all products in df_transactions exist in df_products
    missing_products = df_transactions[~df_transactions[key_column].isin(df_products[key_column])][key_column].unique()
    
    # Check for any rows where the merge resulted in missing values
    missing_values = merged_df.isnull().sum().sum()  # Count total missing values
    
    # Print or return results
    if len(missing_products) == 0 and missing_values == 0:
        print("Merge successful! No data was lost.")
        return True
    else:
        print(f"Warning: {len(missing_products)} products in transactions were not found in products.")
        if len(missing_products) > 0:
            print("Missing Products:", missing_products)
        print(f"Total missing values after merge: {missing_values}")
        return False

# Usage
is_merge_valid = validate_merge(df_transactions, df_Products, enriched_data)

Total missing values after merge: 53


In [336]:
#Check if no data missing after mergin
print(df.isnull().sum())  # Check for nulls in any column
print(f"Rows in Products DataFrame: {Products.shape[0]}")
print(f"Rows in Transactions DataFrame: {Transactions.shape[0]}")
print(f"Rows in Enriched Data: {df.shape[0]}")
print(df.columns)
print(df['Product'].value_counts())

Product                 0
Discount Band          53
Units Sold              0
Sale Price              0
Gross Sales             0
Discounts               0
Sales                   0
COGS                    0
Profit                  0
Date                    0
Month Number            0
ProductID               0
Category                0
Manufacturing Price     0
Segment                 0
Country                 0
dtype: int64
Rows in Products DataFrame: 6
Rows in Transactions DataFrame: 700
Rows in Enriched Data: 700
Index(['Product', 'Discount Band', 'Units Sold', 'Sale Price', 'Gross Sales',
       'Discounts', 'Sales', 'COGS', 'Profit', 'Date', 'Month Number',
       'ProductID', 'Category', 'Manufacturing Price', 'Segment', 'Country'],
      dtype='object')
Product
Paseo        202
Velo         109
VTT          109
Amarilla      94
Carretera     93
Montana       93
Name: count, dtype: int64


4.txt תשובה לשאלה 3 בקובץ

טיפול בנתונים : 


נתונים כספיים


1.deal with the format of the  financial details

In [337]:

# Define a function to clean the financial fields
def clean_financial_field(value):
    if isinstance(value, (int, float)):  # If already a number, return it as is
        return value
    # Remove non-numeric characters (e.g., $, commas, extra dots)
    clean_value = (
        value.replace("$", "")
             .replace(",", "")
             .replace(".", "", value.count(".") - 1)  # Handle decimal points correctly
    )
    try:
        return float(clean_value)  # Convert to a float for analysis
    except ValueError:
        return None  # Return None for non-convertible values
    
# Identify the financial fields that need cleanup
financial_fields = ["Sale Price", "Gross Sales", "Discounts", "Sales", "COGS", "Profit","Manufacturing Price"]

# Apply the cleaning function to each field
for field in financial_fields:
    df[field] = df[field].apply(clean_financial_field)
    
output_filename = "financial_fields.csv"
df.to_csv(output_filename, index=False) 

5.txt תשובה לשאלה 2 בקובץ

עמודת _Segment_Country:


טיפול בנתוני התאריך


העמודה תאריך בטבלה זו מייצגת את העסקה או את תאריך המכירה שבו התרחשה מכירה מסוימת. הוא מתעד את היום, החודש והשנה המדויקים של המכירה.

2,3.Format Date Column and after it formated i create two more columns ,month_name,and year.
also i print the wrong/unusual dates

In [338]:

# Format Date Column
def validate_and_fix_date(row):
    try:
        # Parse the existing date
        date = pd.to_datetime(row['Date'], format='%m/%d/%Y', errors='coerce')
        
        # Check if the date's month matches the Month Number column
        if date and date.month != row['Month Number']:
            print(date.month , row['Month Number'])
            # Correct the date by aligning it with the Month Number column
            corrected_date = pd.Timestamp(year=date.year, month=row['Month Number'], day=1)
        else:
            corrected_date = date
    except:
        # Handle invalid or unusual dates
        print(row['Date'])
        

    return corrected_date

# Identify unusual or missing dates
df['Date'] = df.apply(validate_and_fix_date, axis=1)

# Format the Corrected Date into a single consistent format
df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')
df['Date'] = pd.to_datetime(df['Date'])
df['Year']=df['Date'].dt.year

df["Month Name"] = df["Month Number"].apply(lambda x: calendar.month_name[x])
# df["Year"] = df["Date"].dt.year

output_filename = "add_year_month.csv"
df.to_csv(output_filename, index=False) 

6.txt תשובה לשלאלה 4 בקובץ

יצירת עמודות מחושבות

1.add to df two columns : Discount Impact,Profit Margin

In [309]:
#create new Discount Impact column
df['Discount Impact'] = df['Discounts'] / df['Sales']
df['Profit Margin'] = (df['Profit'] / df['Sales']) * 100
output_filename = "create_two_new_columns.csv"
df.to_csv(output_filename, index=False)

2.create 3 metrics base on the data, the explanation about the metrics is on 7.txt file

In [310]:
# Metric 1: Revenue Contribution
df['Revenue Contribution (%)'] = (df['Gross Sales'] / df['Gross Sales'].sum()) * 100
# Metric 2: Discount Effectiveness
# Avoid dividing by zero for entries with no discounts
df['Discount Effectiveness'] = df.apply(
    lambda row: row['Profit'] / row['Discounts'] if row['Discounts'] != 0 else None, axis=1
)
df['Cost-to-Sales Ratio (%)'] = (df['COGS'] / df['Sales']) * 100
output_filename = "create_three_metrics.csv"
df.to_csv(output_filename, index=False)

יצירת טבלאות מסכמות בעזרת פיווטים 

טבלת מוצר (Product):


In [311]:

# Pivot Table for Product
product_table = pd.pivot_table(
    df,
    values=["Profit Margin", "Discount Impact", "Profit", "Sales"],
    index="Product",
    aggfunc={
        "Profit Margin": "mean",
        "Discount Impact": "mean",
        "Profit": "sum",
        "Sales": "sum"
    }
).rename(columns={
    "Profit Margin": "Avg_Profit_Margin",
    "Discount Impact": "Avg_Discount_Impact",
    "Profit": "Total_Profit",
    "Sales": "Total_Sales"
})
product_table["Total_Transactions"] = df.groupby("Product")["Units Sold"].count()
output_filename = "product_table_using_pivot.csv"
product_table.to_csv(output_filename, index=False)

טבלת סגמנט (Segment):

In [312]:
# Pivot Table for Segment
segment_table = pd.pivot_table(
    df,
    values=["Profit Margin", "Discount Impact", "Profit", "Sales"],
    index="Segment",
    aggfunc={
        "Profit Margin": "mean",
        "Discount Impact": "mean",
        "Profit": "sum",
        "Sales": "sum"
    }
).rename(columns={
    "Profit Margin": "Avg_Profit_Margin",
    "Discount Impact": "Avg_Discount_Impact",
    "Profit": "Total_Profit",
    "Sales": "Total_Sales"
})
segment_table["Total_Transactions"] = df.groupby("Segment")["Units Sold"].count()
output_filename = "segment_table_using_pivot.csv"
segment_table.to_csv(output_filename, index=False)

טבלת אזור גיאוגרפי (Country):


In [313]:
# Pivot Table for Country
country_table = pd.pivot_table(
    df,
    values=["Profit Margin", "Discount Impact", "Profit", "Sales"],
    index="Country",
    aggfunc={
        "Profit Margin": "mean",
        "Discount Impact": "mean",
        "Profit": "sum",
        "Sales": "sum",
    }
).rename(columns={
    "Profit Margin": "Avg_Profit_Margin",
    "Discount Impact": "Avg_Discount_Impact",
    "Profit": "Total_Profit",
    "Sales": "Total_Sales"
})
country_table["Total_Transactions"] = df.groupby("Country")["Units Sold"].count()
country_table = country_table.reset_index()

output_filename = "country_table_using_pivot.csv"
country_table.to_csv(output_filename, index=False)

זיהוי מגמות לאורך זמן 


ניתוח מגמות לפי חודשים (Monthly Trends):

In [314]:
# Monthly Trends Analysis
monthly_trends = df.groupby('Month Number').agg(
    Total_Sales=('Sales', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Avg_Discount_Impact=('Discount Impact', 'mean'),
    Avg_Profit_Margin=('Profit Margin', 'mean')
).reset_index()
output_filename = "monthly_trends.csv"
monthly_trends.to_csv(output_filename, index=False)

ניתוח מגמות לפי שנים (Yearly Trends):


In [315]:
# Annual Trends Analysis
annual_trends = df.groupby('Year').agg(
    Total_Sales=('Sales', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Avg_Discount_Impact=('Discount Impact', 'mean'),
    Avg_Profit_Margin=('Profit Margin', 'mean')
).reset_index()
output_filename = "annual_trends.csv"
annual_trends.to_csv(output_filename, index=False)

ניתוח של מגמות רבעוניות ,הסבר נמצא בקובץ 8 והקוד למטה מייצר את הטבלה 

In [316]:

# Create a Quarter column
df['Quarter'] = pd.to_datetime(df['Date']).dt.quarter

# Group by Quarter and Product to calculate quarterly trends
quarterly_trends = df.groupby(['Quarter', 'Product']).agg(
    Total_Sales=('Sales', 'sum'),
    Avg_Profit_Margin=('Profit Margin', 'mean'),
    Total_Discounts=('Discounts', 'sum'),
    Total_Profit=('Profit', 'sum')
).reset_index()

# Rename columns for clarity
quarterly_trends = quarterly_trends.rename(columns={
    'Total_Sales': 'Total_Quarterly_Sales',
    'Avg_Profit_Margin': 'Avg_Quarterly_Profit_Margin',
    'Total_Discounts': 'Total_Quarterly_Discounts',
    'Total_Profit': 'Total_Quarterly_Profit'
})

output_filename = "quarterly_trends.csv"
quarterly_trends.to_csv(output_filename, index=False)

יצירת פילטרים מותאמים לניתוח ממוקד ויזהוי בעיות עסקיות


זיהוי מוצרים בעייתיים


In [ ]:
# Define the thresholds for underperforming products
low_profit_threshold = 0.07  # Profit Margin < 7%
high_discount_threshold = 0.085  # Discount Impact > 8.5% (Top Quartile)

In [ ]:
output_filename = "underperforming_products_before.csv"
df.to_csv(output_filename, index=False)
# Filter the data based on criteria
underperforming_products = df[
    (df['Profit Margin'] < low_profit_threshold) &
    (df['Discount Impact'] > high_discount_threshold)
]

In [ ]:
output_filename = "underperforming_products.csv"
underperforming_products.to_csv(output_filename, index=False)

In [ ]:
# Select and sort the required columns
underperforming_products = underperforming_products[[
    'Product', 'Segment', 'Sales', 'Profit', 'Discount Impact', 'Profit Margin'
]].sort_values(by='Profit Margin', ascending=True)

In [ ]:
output_filename = "underperforming_products.csv"
underperforming_products.to_csv(output_filename, index=False)

In [ ]:
# Rename columns for better readability
underperforming_products = underperforming_products.rename(columns={
    'Sales': 'Total Sales'
})